In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="""Create an effective prompt template for getting flights""")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Create an effective prompt template for getting flights', additional_kwargs={}, response_metadata={}, id='f8e04033-f65e-48db-8cab-5cc4873863cf'),
              AIMessage(content='**Flight‑Search Prompt Template**\n\nUse the following JSON structure to request flight data from the Kiwi API.  \nReplace the placeholder values (`<…>`) with the user’s preferences.\n\n```json\n{\n  "cabinClass": "<M|W|C|F>",          // M=Economy, W=Economy Premium, C=Business, F=First\n  "curr": "<ISO‑4217 currency code>", // e.g., EUR, USD, GBP\n  "departureDate": "<dd/mm/yyyy>",     // e.g., 15/07/2026\n  "departureDateFlexRange": <0‑3>,     // days before/after the departure date\n  "flyFrom": "<city or airport code>", // e.g., "London" or "LHR"\n  "flyTo": "<city or airport code>",   // e.g., "Paris" or "CDG"\n  "locale": "<en|de|fr|es|it|ru|…>",   // language for city names & links\n  "passengers": {\n    "adults": <1‑9>,\n    "children": <0‑9>,\n    "infants": <0‑9>

In [7]:
print(response["messages"][-1].content)

**Flight‑Search Prompt Template**

Use the following JSON structure to request flight data from the Kiwi API.  
Replace the placeholder values (`<…>`) with the user’s preferences.

```json
{
  "cabinClass": "<M|W|C|F>",          // M=Economy, W=Economy Premium, C=Business, F=First
  "curr": "<ISO‑4217 currency code>", // e.g., EUR, USD, GBP
  "departureDate": "<dd/mm/yyyy>",     // e.g., 15/07/2026
  "departureDateFlexRange": <0‑3>,     // days before/after the departure date
  "flyFrom": "<city or airport code>", // e.g., "London" or "LHR"
  "flyTo": "<city or airport code>",   // e.g., "Paris" or "CDG"
  "locale": "<en|de|fr|es|it|ru|…>",   // language for city names & links
  "passengers": {
    "adults": <1‑9>,
    "children": <0‑9>,
    "infants": <0‑9>
  },
  "returnDate": "<dd/mm/yyyy>",        // optional for one‑way; use same format
  "returnDateFlexRange": <0‑3>,        // optional for one‑way
  "sort": "<price|duration|quality|date>" // default is date
}
```

**Example (Roun

In [ ]:
prompt = """
{
  "cabinClass": "M",
  "curr": "EUR",
  "departureDate": "15/07/2026",
  "departureDateFlexRange": 1,
  "flyFrom": "London",
  "flyTo": "Paris",
  "locale": "en",
  "passengers": {
    "adults": 2,
    "children": 0,
    "infants": 0
  },
  "returnDate": "22/07/2026",
  "returnDateFlexRange": 1,
  "sort": "price"
}
"""

response = await agent.ainvoke(
    {"messages": [HumanMessage(content=prompt)]},
    config
    )

pprint(response)

In [9]:
print(response["messages"][-1].content)

I’m sorry, but I couldn’t find a matching location for “New York JFK” or “Tokyo NRT” in the Kiwi database. Please double‑check the airport names or codes and try again.


In [ ]:
prompt = """
{
  "cabinClass": "C",
  "curr": "USD",
  "departureDate": "02/05/2026",
  "departureDateFlexRange": 0,
  "flyFrom": "LHR",
  "flyTo": "JFK",
  "locale": "en",
  "passengers": {
    "adults": 1,
    "children": 0,
    "infants": 0
  },
  "returnDate": "",
  "returnDateFlexRange": 0,
  "sort": "duration"
}
"""

response = await agent.ainvoke(
    {"messages": [HumanMessage(content=prompt)]},
    config
    )

pprint(response)

In [ ]:
print(response["messages"][-1].content)